In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2025/SAMPLE_SUBMISSION.xlsx
/kaggle/input/widsdatathon2025/Data Dictionary.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAINING_SOLUTIONS.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_CATEGORICAL_METADATA.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_QUANTITATIVE_METADATA.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx
/kaggle/input/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv
/kaggle/input/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx
/kaggle/input/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx


In [2]:
!pip install pandas numpy scikit-learn imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [3]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Modeling
from sklearn.ensemble import RandomForestClassifier

# Resampling
from imblearn.over_sampling import SMOTE

# Cross-validation and tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, ParameterGrid

# Metrics
from sklearn.metrics import f1_score, make_scorer

In [4]:
TRAIN_CATEGORICAL=pd.read_excel('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx')

In [5]:
TRAIN_QUANTITATIVE=pd.read_excel('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx')

In [6]:
dfc=TRAIN_CATEGORICAL
dfq=TRAIN_QUANTITATIVE

In [7]:
dfc.isnull().sum()

participant_id                        0
Basic_Demos_Enroll_Year               0
Basic_Demos_Study_Site                0
PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
dtype: int64

In [8]:
df1=dfc
df1

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,00aIpNTbG5uh,2019,4,1.0,0.0,3.0,21.0,45.0,NaN,NaN
1,00fV0OyyoLfw,2017,1,0.0,9.0,2.0,21.0,0.0,21.0,45.0
2,04X1eiS79T4B,2017,1,1.0,2.0,2.0,9.0,0.0,NaN,NaN
3,05ocQutkURd6,2018,1,3.0,8.0,2.0,18.0,10.0,18.0,0.0
4,06YUNBA9ZRLq,2018,1,0.0,1.0,2.0,12.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,2019,4,1.0,1.0,3.0,12.0,NaN,15.0,5.0
1209,zwXD5v17Rx01,2018,1,0.0,0.0,3.0,21.0,40.0,21.0,40.0
1210,zWzLCi3NTBTd,2018,3,2.0,3.0,3.0,21.0,40.0,21.0,35.0
1211,Zy9GTHDxUbXU,2019,4,0.0,1.0,3.0,18.0,35.0,18.0,45.0


In [9]:
df1.isnull().sum()

participant_id                        0
Basic_Demos_Enroll_Year               0
Basic_Demos_Study_Site                0
PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
dtype: int64

In [10]:
# Fill missing values in all columns with their mode [CATEGORICAL]
for col in df1.columns:
    df1[col] = df1[col].fillna(df1[col].mode()[0])


In [11]:
df1.isnull().sum()

participant_id                      0
Basic_Demos_Enroll_Year             0
Basic_Demos_Study_Site              0
PreInt_Demos_Fam_Child_Ethnicity    0
PreInt_Demos_Fam_Child_Race         0
MRI_Track_Scan_Location             0
Barratt_Barratt_P1_Edu              0
Barratt_Barratt_P1_Occ              0
Barratt_Barratt_P2_Edu              0
Barratt_Barratt_P2_Occ              0
dtype: int64

In [12]:
dfq.isnull().sum()

participant_id                  0
EHQ_EHQ_Total                  13
ColorVision_CV_Score           23
APQ_P_APQ_P_CP                 12
APQ_P_APQ_P_ID                 12
APQ_P_APQ_P_INV                12
APQ_P_APQ_P_OPD                12
APQ_P_APQ_P_PM                 12
APQ_P_APQ_P_PP                 12
SDQ_SDQ_Conduct_Problems        9
SDQ_SDQ_Difficulties_Total      9
SDQ_SDQ_Emotional_Problems      9
SDQ_SDQ_Externalizing           9
SDQ_SDQ_Generating_Impact       9
SDQ_SDQ_Hyperactivity           9
SDQ_SDQ_Internalizing           9
SDQ_SDQ_Peer_Problems           9
SDQ_SDQ_Prosocial               9
MRI_Track_Age_at_Scan         360
dtype: int64

In [13]:
df=dfq
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
1,00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,NaN
2,04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
3,05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
4,06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,100.00,14.0,3.0,14.0,36.0,18.0,13.0,22.0,5.0,20.0,5.0,13.0,5.0,8.0,7.0,2.0,8.0,11.272986
1209,zwXD5v17Rx01,46.67,14.0,3.0,18.0,42.0,21.0,15.0,25.0,2.0,5.0,0.0,5.0,0.0,3.0,0.0,0.0,7.0,7.364020
1210,zWzLCi3NTBTd,-40.00,14.0,3.0,19.0,34.0,20.0,20.0,25.0,2.0,14.0,3.0,9.0,3.0,7.0,5.0,2.0,9.0,9.962126
1211,Zy9GTHDxUbXU,100.00,14.0,3.0,9.0,45.0,23.0,11.0,30.0,1.0,10.0,0.0,8.0,5.0,7.0,2.0,2.0,7.0,NaN


In [14]:
skewness = df.skew(numeric_only=True)

# Loop through each column
for column in df.columns:
    if df[column].isnull().sum() > 0:  # Only process columns with NaNs
        skew_val = skewness[column]

        if -0.5 <= skew_val <= 0.5:
            # Normal distribution — use mean
            df[column] = df[column].fillna(df[column].mean())
            print(f"Filled '{column}' with MEAN (skew={skew_val:.2f})")
        else:
            # Skewed distribution — use median
            df[column] = df[column].fillna(df[column].median())
            print(f"Filled '{column}' with MEDIAN (skew={skew_val:.2f})")

Filled 'EHQ_EHQ_Total' with MEDIAN (skew=-1.67)
Filled 'ColorVision_CV_Score' with MEDIAN (skew=-4.99)
Filled 'APQ_P_APQ_P_CP' with MEDIAN (skew=1.90)
Filled 'APQ_P_APQ_P_ID' with MEAN (skew=0.16)
Filled 'APQ_P_APQ_P_INV' with MEAN (skew=-0.24)
Filled 'APQ_P_APQ_P_OPD' with MEAN (skew=0.04)
Filled 'APQ_P_APQ_P_PM' with MEDIAN (skew=0.91)
Filled 'APQ_P_APQ_P_PP' with MEDIAN (skew=-0.52)
Filled 'SDQ_SDQ_Conduct_Problems' with MEDIAN (skew=1.00)
Filled 'SDQ_SDQ_Difficulties_Total' with MEAN (skew=0.32)
Filled 'SDQ_SDQ_Emotional_Problems' with MEDIAN (skew=0.93)
Filled 'SDQ_SDQ_Externalizing' with MEAN (skew=0.19)
Filled 'SDQ_SDQ_Generating_Impact' with MEAN (skew=0.17)
Filled 'SDQ_SDQ_Hyperactivity' with MEAN (skew=-0.15)
Filled 'SDQ_SDQ_Internalizing' with MEDIAN (skew=0.72)
Filled 'SDQ_SDQ_Peer_Problems' with MEDIAN (skew=0.80)
Filled 'SDQ_SDQ_Prosocial' with MEDIAN (skew=-0.85)
Filled 'MRI_Track_Age_at_Scan' with MEDIAN (skew=0.54)


In [15]:
df.isnull().sum()

participant_id                0
EHQ_EHQ_Total                 0
ColorVision_CV_Score          0
APQ_P_APQ_P_CP                0
APQ_P_APQ_P_ID                0
APQ_P_APQ_P_INV               0
APQ_P_APQ_P_OPD               0
APQ_P_APQ_P_PM                0
APQ_P_APQ_P_PP                0
SDQ_SDQ_Conduct_Problems      0
SDQ_SDQ_Difficulties_Total    0
SDQ_SDQ_Emotional_Problems    0
SDQ_SDQ_Externalizing         0
SDQ_SDQ_Generating_Impact     0
SDQ_SDQ_Hyperactivity         0
SDQ_SDQ_Internalizing         0
SDQ_SDQ_Peer_Problems         0
SDQ_SDQ_Prosocial             0
MRI_Track_Age_at_Scan         0
dtype: int64

In [16]:
# Function to detect outliers using IQR
def detect_outliers_iqr(dfq):
    outliers = {}
    for col in dfq.select_dtypes(include=['number']).columns:  # Apply only to numerical columns
        Q1 = dfq[col].quantile(0.25)  # First quartile (25th percentile)
        Q3 = dfq[col].quantile(0.75)  # Third quartile (75th percentile)
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = dfq[(df[col] < lower_bound) | (dfq[col] > upper_bound)][col]
    return outliers

# Find outliers in the dataset
outliers_dict = detect_outliers_iqr(dfq)

# Print outliers for each column
for col, outliers in outliers_dict.items():
    print(f"\n🔹 Outliers in {col}:")
    print(outliers)
    print("-" * 50)


🔹 Outliers in EHQ_EHQ_Total:
5      -100.00
20      -40.00
30     -100.00
43      -44.47
46      -33.35
         ...  
1175    -60.00
1196    -86.67
1199   -100.00
1202    -66.67
1210    -40.00
Name: EHQ_EHQ_Total, Length: 110, dtype: float64
--------------------------------------------------

🔹 Outliers in ColorVision_CV_Score:
0       13.0
16      13.0
22      13.0
32       0.0
35      13.0
        ... 
1189    13.0
1191    11.0
1198    13.0
1203    13.0
1207    13.0
Name: ColorVision_CV_Score, Length: 221, dtype: float64
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_CP:
4       8.0
21      6.0
30      6.0
38      6.0
66      7.0
       ... 
1162    6.0
1192    7.0
1195    6.0
1199    6.0
1201    8.0
Name: APQ_P_APQ_P_CP, Length: 127, dtype: float64
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_ID:
220    24.0
272    24.0
777    28.0
Name: APQ_P_APQ_P_ID, dtype: float64
--------------------------------------------------

In [17]:
# Function to cap outliers using IQR method-------
def cap_outliers(dfq):
    for col in dfq.select_dtypes(include=['number']).columns:
        Q1 = dfq[col].quantile(0.25)
        Q3 = dfq[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Replace outliers with the respective bounds
        df[col] = np.where(dfq[col] < lower_bound, lower_bound, dfq[col])
        df[col] = np.where(dfq[col] > upper_bound, upper_bound, dfq[col])

    return df

# Fix the outliers in the dataset
df_fixed = cap_outliers(dfq)

# Verify if outliers are fixed
print("\n✅ Outliers have been capped. Here are the updated value counts:")
for col in df_fixed.select_dtypes(include=['number']).columns:
    print(f"\nValue Counts for Column: {col}")
    print(df_fixed[col].value_counts().sort_index())
    print("-" * 50)


✅ Outliers have been capped. Here are the updated value counts:

Value Counts for Column: EHQ_EHQ_Total
EHQ_EHQ_Total
-23.425     110
-23.330       1
-20.000       6
-16.680       3
-14.470       1
           ... 
 94.470      12
 95.600       7
 96.670      18
 97.800      14
 100.000    252
Name: count, Length: 123, dtype: int64
--------------------------------------------------

Value Counts for Column: ColorVision_CV_Score
ColorVision_CV_Score
14.0    1213
Name: count, dtype: int64
--------------------------------------------------

Value Counts for Column: APQ_P_APQ_P_CP
APQ_P_APQ_P_CP
2.0      3
3.0    752
4.0    177
5.0    154
5.5    127
Name: count, dtype: int64
--------------------------------------------------

Value Counts for Column: APQ_P_APQ_P_ID
APQ_P_APQ_P_ID
5.000000       1
6.000000      31
7.000000      21
8.000000      58
9.000000      79
10.000000     94
11.000000     89
12.000000    122
13.000000    129
13.337219     12
14.000000    133
15.000000    104
16.000000

In [18]:
# Find outliers in the dataset
outliers_dict = detect_outliers_iqr(dfq)

# Print outliers for each column
for col, outliers in outliers_dict.items():
    print(f"\n🔹 Outliers in {col}:")
    print(outliers)
    print("-" * 50)


🔹 Outliers in EHQ_EHQ_Total:
Series([], Name: EHQ_EHQ_Total, dtype: float64)
--------------------------------------------------

🔹 Outliers in ColorVision_CV_Score:
Series([], Name: ColorVision_CV_Score, dtype: float64)
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_CP:
Series([], Name: APQ_P_APQ_P_CP, dtype: float64)
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_ID:
Series([], Name: APQ_P_APQ_P_ID, dtype: float64)
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_INV:
Series([], Name: APQ_P_APQ_P_INV, dtype: float64)
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_OPD:
Series([], Name: APQ_P_APQ_P_OPD, dtype: float64)
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_PM:
Series([], Name: APQ_P_APQ_P_PM, dtype: float64)
--------------------------------------------------

🔹 Outliers in APQ_P_APQ_P_PP:
Series([], Name: APQ_P_APQ_P_PP, dtype

In [19]:
x_train=pd.merge(df_fixed, df1, on='participant_id', how='inner')
x_train

,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,...,MRI_Track_Age_at_Scan,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,00aIpNTbG5uh,100.000,14.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,...,14.274127,2019,4,1.0,0.0,3.0,21.0,45.0,21.0,45.0
1,00fV0OyyoLfw,92.270,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,...,10.739219,2017,1,0.0,9.0,2.0,21.0,0.0,21.0,45.0
2,04X1eiS79T4B,86.670,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,...,13.463381,2017,1,1.0,2.0,2.0,9.0,0.0,21.0,45.0
3,05ocQutkURd6,93.340,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,...,9.572553,2018,1,3.0,8.0,2.0,18.0,10.0,18.0,0.0
4,06YUNBA9ZRLq,0.000,14.0,5.5,12.0,35.0,22.0,12.0,24.0,6.0,...,6.654574,2018,1,0.0,1.0,2.0,12.0,0.0,21.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,100.000,14.0,3.0,14.0,36.0,18.0,13.0,22.0,5.0,...,11.272986,2019,4,1.0,1.0,3.0,12.0,0.0,15.0,5.0
1209,zwXD5v17Rx01,46.670,14.0,3.0,18.0,42.0,21.0,15.0,25.0,2.0,...,7.364020,2018,1,0.0,0.0,3.0,21.0,40.0,21.0,40.0
1210,zWzLCi3NTBTd,-23.425,14.0,3.0,19.0,34.0,20.0,20.0,25.0,2.0,...,9.962126,2018,3,2.0,3.0,3.0,21.0,40.0,21.0,35.0
1211,Zy9GTHDxUbXU,100.000,14.0,3.0,9.0,45.0,23.0,11.0,30.0,1.0,...,10.739219,2019,4,0.0,1.0,3.0,18.0,35.0,18.0,45.0


X Test Preprocessing

In [20]:
xcat=pd.read_excel('/kaggle/input/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx')
xquant=pd.read_excel('/kaggle/input/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx')

In [21]:
xcat.isnull().sum()

participant_id                       0
Basic_Demos_Enroll_Year              0
Basic_Demos_Study_Site               0
PreInt_Demos_Fam_Child_Ethnicity     3
PreInt_Demos_Fam_Child_Race          6
MRI_Track_Scan_Location              0
Barratt_Barratt_P1_Edu               1
Barratt_Barratt_P1_Occ               1
Barratt_Barratt_P2_Edu              36
Barratt_Barratt_P2_Occ              42
dtype: int64

In [22]:
xquant.isnull().sum()

participant_id                 0
EHQ_EHQ_Total                  1
ColorVision_CV_Score           9
APQ_P_APQ_P_CP                15
APQ_P_APQ_P_ID                15
APQ_P_APQ_P_INV               15
APQ_P_APQ_P_OPD               15
APQ_P_APQ_P_PM                15
APQ_P_APQ_P_PP                15
SDQ_SDQ_Conduct_Problems      30
SDQ_SDQ_Difficulties_Total    30
SDQ_SDQ_Emotional_Problems    30
SDQ_SDQ_Externalizing         30
SDQ_SDQ_Generating_Impact     30
SDQ_SDQ_Hyperactivity         30
SDQ_SDQ_Internalizing         30
SDQ_SDQ_Peer_Problems         30
SDQ_SDQ_Prosocial             30
MRI_Track_Age_at_Scan          0
dtype: int64

X Test Quantitative:  Handling Nan Values

In [23]:
df=xquant
skewness = df.skew(numeric_only=True)

# Loop through each column
for column in df.columns:
    if df[column].isnull().sum() > 0:  # Only process columns with NaNs
        skew_val = skewness[column]

        if -0.5 <= skew_val <= 0.5:
            # Normal distribution — use mean
            df[column] = df[column].fillna(df[column].mean())
            print(f"Filled '{column}' with MEAN (skew={skew_val:.2f})")
        else:
            # Skewed distribution — use median
            df[column] = df[column].fillna(df[column].median())
            print(f"Filled '{column}' with MEDIAN (skew={skew_val:.2f})")

Filled 'EHQ_EHQ_Total' with MEDIAN (skew=-1.77)
Filled 'ColorVision_CV_Score' with MEDIAN (skew=-4.86)
Filled 'APQ_P_APQ_P_CP' with MEDIAN (skew=2.90)
Filled 'APQ_P_APQ_P_ID' with MEAN (skew=0.06)
Filled 'APQ_P_APQ_P_INV' with MEAN (skew=-0.32)
Filled 'APQ_P_APQ_P_OPD' with MEAN (skew=0.24)
Filled 'APQ_P_APQ_P_PM' with MEDIAN (skew=0.85)
Filled 'APQ_P_APQ_P_PP' with MEDIAN (skew=-0.58)
Filled 'SDQ_SDQ_Conduct_Problems' with MEDIAN (skew=1.14)
Filled 'SDQ_SDQ_Difficulties_Total' with MEAN (skew=0.28)
Filled 'SDQ_SDQ_Emotional_Problems' with MEDIAN (skew=0.88)
Filled 'SDQ_SDQ_Externalizing' with MEAN (skew=0.17)
Filled 'SDQ_SDQ_Generating_Impact' with MEAN (skew=0.30)
Filled 'SDQ_SDQ_Hyperactivity' with MEAN (skew=-0.27)
Filled 'SDQ_SDQ_Internalizing' with MEDIAN (skew=0.70)
Filled 'SDQ_SDQ_Peer_Problems' with MEDIAN (skew=1.01)
Filled 'SDQ_SDQ_Prosocial' with MEDIAN (skew=-0.93)


In [24]:
df.isnull().sum()

participant_id                0
EHQ_EHQ_Total                 0
ColorVision_CV_Score          0
APQ_P_APQ_P_CP                0
APQ_P_APQ_P_ID                0
APQ_P_APQ_P_INV               0
APQ_P_APQ_P_OPD               0
APQ_P_APQ_P_PM                0
APQ_P_APQ_P_PP                0
SDQ_SDQ_Conduct_Problems      0
SDQ_SDQ_Difficulties_Total    0
SDQ_SDQ_Emotional_Problems    0
SDQ_SDQ_Externalizing         0
SDQ_SDQ_Generating_Impact     0
SDQ_SDQ_Hyperactivity         0
SDQ_SDQ_Internalizing         0
SDQ_SDQ_Peer_Problems         0
SDQ_SDQ_Prosocial             0
MRI_Track_Age_at_Scan         0
dtype: int64

X Categorical: Filling NAN

In [25]:
# Fill missing values in all columns with their mode [CATEGORICAL]
df1=xcat
for col in df1.columns:
    df1[col] = df1[col].fillna(df1[col].mode()[0])

In [26]:
df1.isnull().sum()

participant_id                      0
Basic_Demos_Enroll_Year             0
Basic_Demos_Study_Site              0
PreInt_Demos_Fam_Child_Ethnicity    0
PreInt_Demos_Fam_Child_Race         0
MRI_Track_Scan_Location             0
Barratt_Barratt_P1_Edu              0
Barratt_Barratt_P1_Occ              0
Barratt_Barratt_P2_Edu              0
Barratt_Barratt_P2_Occ              0
dtype: int64

In [27]:
mergeddf = pd.merge(df, df1, on='participant_id', how='inner')

In [28]:
xtest=mergeddf

In [29]:
x_train.shape

(1213, 28)

In [30]:
xtest.shape

(304, 28)

Feature Selection : XTrain

In [31]:
x_train.columns

Index(['participant_id', 'EHQ_EHQ_Total', 'ColorVision_CV_Score',
       'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV',
       'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP',
       'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
       'MRI_Track_Age_at_Scan', 'Basic_Demos_Enroll_Year',
       'Basic_Demos_Study_Site', 'PreInt_Demos_Fam_Child_Ethnicity',
       'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location',
       'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ',
       'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ'],
      dtype='object')

In [32]:
partid=x_train['participant_id']
partid

0       00aIpNTbG5uh
1       00fV0OyyoLfw
2       04X1eiS79T4B
3       05ocQutkURd6
4       06YUNBA9ZRLq
            ...     
1208    zwjJWCRzKhDz
1209    zwXD5v17Rx01
1210    zWzLCi3NTBTd
1211    Zy9GTHDxUbXU
1212    Zye7yYRQohXi
Name: participant_id, Length: 1213, dtype: object

In [33]:
xtrainf=x_train[['participant_id','EHQ_EHQ_Total',
        'APQ_P_APQ_P_INV',
       'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM',
        'SDQ_SDQ_Difficulties_Total',
       'SDQ_SDQ_Emotional_Problems',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing']]

In [34]:
xtrainf

,participant_id,EHQ_EHQ_Total,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing
0,00aIpNTbG5uh,100.000,44.0,14.0,20.0,17.0,4.0,5.0,8.0,6.0
1,00fV0OyyoLfw,92.270,35.0,25.0,28.0,20.0,4.0,5.0,8.0,7.0
2,04X1eiS79T4B,86.670,37.0,18.0,26.0,24.0,7.0,10.0,7.0,14.0
3,05ocQutkURd6,93.340,42.0,15.0,20.0,5.0,0.0,0.0,3.0,2.0
4,06YUNBA9ZRLq,0.000,35.0,22.0,12.0,23.0,7.0,8.0,9.0,8.0
...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,100.000,36.0,18.0,13.0,20.0,5.0,5.0,8.0,7.0
1209,zwXD5v17Rx01,46.670,42.0,21.0,15.0,5.0,0.0,0.0,3.0,0.0
1210,zWzLCi3NTBTd,-23.425,34.0,20.0,20.0,14.0,3.0,3.0,7.0,5.0
1211,Zy9GTHDxUbXU,100.000,45.0,23.0,11.0,10.0,0.0,5.0,7.0,2.0


In [35]:
dfmri = pd.read_csv('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
merged_df = pd.merge(xtrainf, dfmri, on='participant_id', how='inner')  # 'inner' can be 'left', 'right', or 'outer' too

In [36]:
dfmri.shape

(1213, 19901)

In [37]:
xtrain=merged_df
xtrain

,participant_id,EHQ_EHQ_Total,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,00aIpNTbG5uh,100.000,44.0,14.0,20.0,17.0,4.0,5.0,8.0,6.0,...,-0.047477,-0.096450,-0.280312,0.037560,0.423037,0.242453,0.336213,0.402338,0.327915,0.539032
1,00fV0OyyoLfw,92.270,35.0,25.0,28.0,20.0,4.0,5.0,8.0,7.0,...,-0.143218,-0.276464,-0.332783,-0.332711,0.556939,0.475578,0.429196,0.457970,0.312571,0.595978
2,04X1eiS79T4B,86.670,37.0,18.0,26.0,24.0,7.0,10.0,7.0,14.0,...,-0.133778,-0.108588,-0.002132,-0.175586,0.679183,0.290292,0.486680,0.255208,0.575017,0.605182
3,05ocQutkURd6,93.340,42.0,15.0,20.0,5.0,0.0,0.0,3.0,2.0,...,-0.096647,-0.041420,-0.199576,-0.216457,0.519074,0.298586,0.415466,0.511607,0.361204,0.446613
4,06YUNBA9ZRLq,0.000,35.0,22.0,12.0,23.0,7.0,8.0,9.0,8.0,...,-0.158995,-0.274017,-0.141012,-0.002865,0.515169,0.336139,0.316430,0.442230,0.177079,0.378278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,100.000,36.0,18.0,13.0,20.0,5.0,5.0,8.0,7.0,...,0.157897,0.122978,0.112789,0.211312,0.601190,0.587116,0.312695,0.485938,0.189102,0.354333
1209,zwXD5v17Rx01,46.670,42.0,21.0,15.0,5.0,0.0,0.0,3.0,0.0,...,0.343204,0.198827,0.253990,0.198741,0.648260,0.055241,0.491985,0.118676,0.404331,0.537121
1210,zWzLCi3NTBTd,-23.425,34.0,20.0,20.0,14.0,3.0,3.0,7.0,5.0,...,0.110834,-0.002346,0.044653,0.234887,0.538475,0.024265,0.472322,0.095624,0.205326,0.182633
1211,Zy9GTHDxUbXU,100.000,45.0,23.0,11.0,10.0,0.0,5.0,7.0,2.0,...,0.013906,0.146094,-0.035955,-0.062152,0.706214,0.183288,0.104987,0.420463,0.152727,0.706737


In [38]:
xtestf=xtest[['participant_id','EHQ_EHQ_Total',
        'APQ_P_APQ_P_INV',
       'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM',
        'SDQ_SDQ_Difficulties_Total',
       'SDQ_SDQ_Emotional_Problems',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing']]
xtestf

,participant_id,EHQ_EHQ_Total,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing
0,Cfwaf5FX7jWK,60.03,41.0,19.0,11.0,12.0,3.0,2.0,7.0,3.0
1,vhGrzmvA3Hjq,86.71,43.0,18.0,15.0,16.0,8.0,7.0,3.0,11.0
2,ULliyEXjy4OV,26.68,36.0,16.0,14.0,7.0,1.0,1.0,5.0,1.0
3,LZfeAb1xMtql,93.38,41.0,17.0,18.0,15.0,4.0,8.0,6.0,5.0
4,EnFOUv0YK1RG,-93.38,42.0,19.0,16.0,18.0,6.0,5.0,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...
299,UadZfjdEg7eG,86.71,41.0,17.0,11.0,18.0,7.0,5.0,6.0,11.0
300,IUEHiLmQAqCi,73.37,38.0,22.0,9.0,16.0,2.0,5.0,9.0,5.0
301,cRySmCadYFRO,87.84,42.0,15.0,10.0,11.0,4.0,4.0,3.0,7.0
302,E3MvDUtJadc5,46.76,43.0,18.0,12.0,21.0,2.0,6.0,5.0,11.0




---


**Merging xtrain and y training solutions**


---









In [39]:
features = xtrain
labels = pd.read_excel('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx')

# Merge the two datasets on the 'participant_id' column (inner join to ensure both features and labels are present)
train = pd.merge(features, labels, on="participant_id", how="inner")

✅ Step (ii): Cross-validation with Stratified K-Fold, Resampling for Sex_F, and Hyperparameter Tuning (RandomForest)

In [40]:
#dropping and selecting features
y_adhd = train["ADHD_Outcome"]
y_sex_f = train["Sex_F"]
X = train.drop(columns=["ADHD_Outcome", "Sex_F", "participant_id"])

In [41]:
#parameter setting
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None],
    'min_samples_split': [2, 5]
}

In [42]:
#cross validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scorer = make_scorer(f1_score, average='weighted')


In [43]:
rf_clf_adhd = RandomForestClassifier(random_state=42)

grid_adhd_rf = GridSearchCV(
    rf_clf_adhd,
    param_grid=param_grid_rf,
    scoring=f1_scorer,
    cv=5,  # You can try cv=3 to speed it up
    n_jobs=-1,
    verbose=1
)

grid_adhd_rf.fit(X, y_adhd)
best_rf_adhd = grid_adhd_rf.best_estimator_
print("Best RF params for ADHD:", grid_adhd_rf.best_params_)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best RF params for ADHD: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


In [44]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, make_scorer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from scipy.stats import randint

pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42))
])

# ---------- Define Hyperparameter Search Space ---------- #
param_dist = {
    'rf__n_estimators': randint(50, 150),
    'rf__max_depth': randint(3, 10),
    'rf__min_samples_split': randint(2, 10),
    'rf__min_samples_leaf': randint(1, 5)
}

# ---------- Scorer ---------- #
scorer = make_scorer(f1_score, average='weighted')

# ---------- Stratified K-Fold ---------- #
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# ---------- Randomized Search ---------- #
search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    scoring=scorer,
    n_iter=15,  # Tries 15 random combinations
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Run in parallel
)

# ---------- Fit the model ---------- #
search.fit(X, y_sex_f)

# ---------- Output Best Results ---------- #
print("\n✅ Best Weighted F1 Score (Sex_F):", search.best_score_)
print("🏆 Best Parameters:", search.best_params_)

# ---------- Final model for predictions ---------- #
best_rf_sex_f = search.best_estimator_


Fitting 3 folds for each of 15 candidates, totalling 45 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



✅ Best Weighted F1 Score (Sex_F): 0.6520618041727553
🏆 Best Parameters: {'rf__max_depth': 3, 'rf__min_samples_leaf': 3, 'rf__min_samples_split': 6, 'rf__n_estimators': 100}


✅ Step (iii): Weighted F1 Score for Each Target (Random Forest)

In [45]:
#for ADHD
f1_scores_adhd_rf = []

train_df = x_train
train_Solutions = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

# Feature matrix and target
X = train_df.drop(columns=["participant_id"])
y_sex_f = train_Solutions["Sex_F"]

for train_idx, val_idx in cv.split(X, y_adhd):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y_adhd.iloc[train_idx], y_adhd.iloc[val_idx]

    model = best_rf_adhd
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='weighted')
    f1_scores_adhd_rf.append(f1)

mean_f1_adhd_rf = np.mean(f1_scores_adhd_rf)
print("Cross-validated Weighted F1 Score for ADHD_Outcome (RF):", mean_f1_adhd_rf)


Cross-validated Weighted F1 Score for ADHD_Outcome (RF): 0.7837100139734526


✅ Step (iv): Average of Weighted F1 Scores (ADHD + Sex_F)

In [46]:
# Retraining ADHD model
final_rf_adhd = best_rf_adhd
final_rf_adhd.fit(X, y_adhd)

RandomForestClassifier(random_state=42)

In [47]:
#Retraining Sex_F with resampling smote
# Resample full data using SMOTE
sm = SMOTE(random_state=42)
X_res_sex_f, y_res_sex_f = sm.fit_resample(X, y_sex_f)

# Retrain using the best model & parameters
final_rf_sex_f = best_rf_sex_f
final_rf_sex_f.fit(X_res_sex_f, y_res_sex_f)

Pipeline(steps=[('smote', SMOTE(random_state=42)),
                ('rf',
                 RandomForestClassifier(max_depth=3, min_samples_leaf=3,
                                        min_samples_split=6,
                                        random_state=42))])

In [48]:
xtestpid=xtest['participant_id']
xtestpid

0      Cfwaf5FX7jWK
1      vhGrzmvA3Hjq
2      ULliyEXjy4OV
3      LZfeAb1xMtql
4      EnFOUv0YK1RG
           ...     
299    UadZfjdEg7eG
300    IUEHiLmQAqCi
301    cRySmCadYFRO
302    E3MvDUtJadc5
303    dQJXfyRazknD
Name: participant_id, Length: 304, dtype: object

✅ Step (vi): Predict on Unlabeled x_test (Random Forest)

In [49]:
# Make sure to exclude participant_id during prediction
#X_test_input = xtest.drop(columns=["participant_id"])  # Remove this line

# Select the same features for X_test_input that were used during training (X)
# Instead of selecting a subset of columns, get all columns except participant_id and the target variables:
#X_test_input = xtest[[col for col in X.columns if col in xtest.columns]] # Remove this line

#Merge with the dfmri dataframe if needed
dfmri = pd.read_csv('/kaggle/input/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')
merged_df = pd.merge(dfmri, xtestf, on='participant_id', how='inner')  # 'inner' can be 'left', 'right', or 'outer' too

# Reassign X_test_input based on the merged dataframe:
# Select the same features for X_test_input that were used during training (X)
# Ensure all columns in X are present in merged_df, and select them in the correct order:
X_test_input = merged_df[[col for col in X.columns if col in merged_df.columns]]

# If some columns are missing in merged_df, add them with NaN values:
missing_cols = [col for col in X.columns if col not in merged_df.columns]
for col in missing_cols:
    merged_df[col] = np.nan  # Fill with NaN for missing columns

# Re-select X_test_input with all columns in the correct order:
X_test_input = merged_df[X.columns]


# Predict
adhd_preds_rf = final_rf_adhd.predict(X_test_input)
sex_f_preds_rf = final_rf_sex_f.predict(X_test_input)

✅ Step (vii): Prepare Submission File (Random Forest)

In [50]:
# Load the Excel file that contains the correct order of participant_id
excel_file = pd.read_excel("/kaggle/input/widsdatathon2025/SAMPLE_SUBMISSION.xlsx")  # Path to the Excel file
correct_order = excel_file["participant_id"]  # Ensure this column matches the Excel file

#  Create a DataFrame for the predictions
submission_rf = pd.DataFrame({
    "participant_id": xtestpid,   # Participant ID from x_test
    "ADHD_Outcome": adhd_preds_rf,
    "Sex_F": sex_f_preds_rf
})

#: Reorder the predictions DataFrame based on the correct order of participant_id
reordered_submission = pd.merge(correct_order, submission_rf, on="participant_id", how="left")

# Save the reordered predictions to a CSV file
reordered_submission.to_csv("submission.csv", index=False)
print("✅ submission saved to 'submission.csv'")


✅ submission saved to 'submission.csv'
